In [1]:
import pickle
from datetime import datetime
import pandas as pd 
import os

In [2]:
def load_pickle(data_path):
    with open(data_path,'rb') as f:
        data = pickle.load(f)
    return data
def save_pickle(data,save_path):
    with open(save_path,'wb') as f:
        pickle.dump(data,f)

## 1. convert to Recbole and SFT format

### 1. Recoble format
- category.pickle
- dataset_name.inter
- dataset_name.item

### 2. SFT format
- category.pickle 'category': [item_ids]
- metas.pickle 'item_id': {'item_id', 'title'}
- sequential.pickle 'user_id': [item_ids]

In [3]:
file_path = '/home/wangshuo/codes/scope-rec/data/toys/toys_final.pickle'
final_df = load_pickle(file_path)
final_df.head()

,reviewerID,interactions,count
182043,AIU50D6HNVAPO,[(VTech Turn and Learn Driver Amazon Exclusive...,18
58848,A22S8MRYUCAQ4W,[(Monster High Toy - Kjersti Trollson Deluxe F...,18
104303,A2WD2F9PU3K60Z,"[(VTech Kidizoom Spin and Smile Camera, Violet...",26
160394,A4LXJHAJZG7GW,[(Fisher-Price Sesame Street Singing Pizza Elm...,49
198833,ATT2UJWZVXOE0,"[(Fisher-Price Little People Disney Pricess, A...",21


In [4]:
final_df.iloc[0]['interactions']

[('VTech Turn and Learn Driver Amazon Exclusive',
  'B012C8L010',
  'AIU50D6HNVAPO',
  1454025600,
  "Great toy. Baby loves it. We use it at home and as a car toy. There's enough buttons and sound options to keep baby entertained but also not be too overwhelming. All in all good purchase",
  ['Toys & Games', "Kids' Electronics", 'Electronic Learning Toys']),
 ('Yellow Mountain Imports Magnetic Travel Chess Set (9.7 Inches) - Portable - Perfectly Travel-Sized - Complete Playing Pieces Included in Set',
  'B0019FATKG',
  'AIU50D6HNVAPO',
  1454025600,
  "Bought this as a stocking stuffer for my son for Christmas. I love the fact that it's magnetic and the board itself is the case for the pieces. Great buy",
  ['Toys & Games', 'Games', 'Travel Games']),
 ('Hello Kitty Figure Necklace Pendant Bowknot Silver',
  'B00A6MRM8U',
  'AIU50D6HNVAPO',
  1454112000,
  'Broke within the first day. Not worth buying.',
  ['Toys & Games',
   'Dress Up & Pretend Play',
   'Beauty & Fashion',
   'Length 

In [5]:
import ast
def convert_to_list(tags_str):
    try:
        tags = ast.literal_eval(tags_str)
        all_tags = []
        for tag in tags:
            all_tags.extend(tag)
        return tags
    except:
        return []

In [6]:

sequential = {}
metas ={}
category = {}
iter =[]
item_recbole = []
item_list =[]
COMMON_TAG = 'Toys&Games'
miss_count = 0
miss_titile =0 
empty_count = 0

for index, row in final_df.iterrows():
    user_id = str(row['reviewerID'])
    # interactions(item_title,item_id, timestamp, tags(list))
    interactions = row['interactions']
    item_ids = []

    for interaction in interactions:
        if type(interaction[0]) != str:
            miss_titile += 1
            continue
        item_title = interaction[0].strip()
        item_id = str(interaction[1])
        tags = interaction[5]
        timestamp = interaction[3]
        
        
        if len(tags) == 0:
            tags = [COMMON_TAG]
            miss_count += 1

        for tag in tags:
            tag =tag.strip()
            if tag not in category:
                category[tag] = []
            if item_id not in category[tag]:
                category[tag].append(item_id)
            # category[tag].append(item_id)
        iter.append({"user_id:token":user_id,"item_id:token":item_id,"timestamp:float":float(timestamp)})
        # item_recbole.append({"item_id:token":item_id,"timestamp:float":toTimestamp(timestamp)})
        item_ids.append(item_id)
        metas[item_id]={"title":item_title}
        item_list.append(item_title)
    
    if len(item_ids) <= 3:
        empty_count += 1
        continue
    sequential[user_id]=item_ids
print('miss_count:',miss_count)
print("miss_titile:",miss_titile)
print("empty_count:",empty_count)

miss_count: 8174
miss_titile: 0
empty_count: 0


In [7]:
len(metas)

49985

In [8]:
iter_df = pd.DataFrame(iter)
iter_df = iter_df[["user_id:token","item_id:token","timestamp:float"]]
iter_df.to_csv("/home/wangshuo/codes/scope-rec/SASRec/dataset/toys/toys.inter",index=False,sep='\t')

In [51]:
save_base_dir = "/home/wangshuo/codes/scope-rec/data/toys/"
save_pickle(sequential,os.path.join(save_base_dir,"sequential.pickle"))
save_pickle(metas,os.path.join(save_base_dir,"metas.pickle"))
save_pickle(category,os.path.join(save_base_dir,"category.pickle"))
save_pickle(item_list,os.path.join(save_base_dir,"items.pickle"))

# data overview

In [49]:
sequential['10158']

['B007U7M0L8', 'B0015ZITNY', '0911121013']

In [48]:
empty_count=0
for key in sequential.keys():
    if len(sequential[key]) <= 2:
        print(key,sequential[key])
        empty_count += 1
print(empty_count)

0


In [4]:
save_base_dir = "/home/wangshuo/codes/InstructControllableRec_RLHF/data/dataset/steam/"
data = load_pickle(os.path.join(save_base_dir,"metas.pickle"))

In [16]:
len(set(category[COMMON_TAG]))

555

In [4]:
data1=load_pickle("/home/wangshuo/codes/scope-rec/data/dataset/steam/metas.pickle")
data1

{'373420': {'title': 'Divinity: Original Sin - Enhanced Edition'},
 '289420': {'title': 'Planet Alcatraz'},
 '475150': {'title': 'Titan Quest Anniversary Edition'},
 '6020': {'title': 'STAR WARS™ Jedi Knight - Jedi Academy™'},
 '671510': {'title': 'DESOLATE'},
 '377160': {'title': 'Fallout 4'},
 '738520': {'title': 'Breathedge'},
 '976620': {'title': 'Enderal: Forgotten Stories (Special Edition)'},
 '379430': {'title': 'Kingdom Come: Deliverance'},
 '829590': {'title': 'CryoFall'},
 '552620': {'title': 'ATOM RPG: Post-apocalyptic indie game'},
 '233860': {'title': 'Kenshi'},
 '365360': {'title': 'Battle Brothers'},
 '285500': {'title': 'Hard Truck Apocalypse / Ex Machina'},
 '1227530': {'title': 'Partisans 1941'},
 '516750': {'title': 'My Summer Car'},
 '214730': {'title': 'Space Rangers HD: A War Apart'},
 '233450': {'title': 'Prison Architect'},
 '431910': {'title': 'Psychocat: The Answer'},
 '399670': {'title': 'Game Corp DX'},
 '313120': {'title': 'Stranded Deep'},
 '327640': {'tit

In [6]:
data2=load_pickle("/home/wangshuo/codes/rec-scope/constraint_search/data/steam/base/items.pickle")

In [7]:
not_in = []
for item in data1:
    if item not in data2:
        not_in.append(item)
print(len(not_in))

0


In [18]:
'The Witcher® 3: Wild Hunt' in data2

True

In [4]:
# 统计一下每个item的平均token个数
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("/home/wangshuo/weights/llama2/Llama-2-7b-hf-chat/")

In [15]:
def get_token_num(text):
    token_list = tokenizer.encode(text,add_special_tokens=False)
    return len(tokenizer.encode(text))

In [8]:
def cal_token_num(data_path):
    data = load_pickle(data_path)
    token_num = 0
    for item in data:
        token_num+= get_token_num(item)
    return token_num/len(data)

In [20]:
data_path = "/home/wangshuo/codes/InstructControllableRec_RLHF/data/dataset/movies/items.pickle"
token_num = cal_token_num(data_path)
print(token_num)

9.18444705882353
